In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns;
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler
import time
import math

sns.set()  
import numpy as np
from collections import defaultdict

from numpy import dot
from numpy.linalg import norm

def lineToTuple(line):
    
    cleanLine = line.strip()
    
    cleanLine = cleanLine.replace('"', '')
    
    lineList = cleanLine.split(",")
    
    stringsToNumbers(lineList)
    lineTuple = tuple(lineList)
    return lineTuple

def stringsToNumbers(myList):
    for i in range(len(myList)):
        if (isValidNumberString(myList[i])):
            myList[i] = float(myList[i])

def isValidNumberString(s):
  if len(s) == 0:
    return False
  if  len(s) > 1 and s[0] == "-":
      s = s[1:]
  for c in s:
    if c not in "0123456789.":
      return False
  return True

def loadCSV(fileName):
    fileHandler = open(fileName, "rt")
    lines = fileHandler.readlines()
    fileHandler.close()
    del lines[0]  
    dataset = []
    for line in lines:
        instance = lineToTuple(line)
        dataset.append(instance)
    return dataset

def euclidean_distance(point1, point2):
    distance = 0
    for a,b in zip(point1, point2):
        distance += pow((a-b), 2)
    return math.sqrt(distance)
def manhattan_distance(point1, point2):
    distance = 0
    for a,b in zip(point1, point2):
        distance += abs(a-b)
    return distance
def cosine_similarity(point1, point2):
  A = np.array(point1)
  B = np.array(point2)
  dist = 1 - np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))
  return dist
def jaccard(A, B):
    return 1 - (np.sum(np.minimum(A,B), axis = 0)/np.sum(np.maximum(A, B), axis = 0)) 
def calculate_centroid(cluster):
  n = len(cluster[0])
  
  centroid = [0]*(n-1)

  for i in range(n-1):
    for point in cluster:
      centroid[i] += point[i]
    centroid[i] = centroid[i]/len(cluster)
  

  
  return centroid
def draw_nd_scatter(clusters, centroid_centers):
  for key in clusters:
    x = []
    y = []
    cluster = clusters[key]
    for c in cluster:
      x.append(c[0])
      y.append(c[1])
    plt.scatter(x, y, marker='*', s=50, cmap='viridis')

  for point in centroid_centers:
    plt.scatter(point[0], point[1], c='black', s=200, alpha=0.5)
  
  plt.show()
def label_cluster(cluster):
  cl = defaultdict(int)
  for point in cluster:
    cl[point[-1]] += 1
  return cl
class KMeans:
  def __init__(self, n_clusters=10, max_iters=100, init_centroids=None, d_func=euclidean_distance, show_sse=False, show_first_centroid=False, centroid_stop=True):
    self.n_clusters = n_clusters
    self.max_iters = max_iters
    self.init_centroids = init_centroids
    self.d_func = d_func
    self.sse_list = []
    self.show_first_centroid = show_first_centroid
    self.show_sse = show_sse
    self.centroid_stop = centroid_stop

  def fit(self, data):
    start = time.time()
    if self.init_centroids is None:
      
      random_choice = np.random.choice(range(len(data)), self.n_clusters, replace=False)
      centroids = []

      for choice in random_choice:
        
        centroids.append(data[choice][:-1])
        
      
      self.init_centroids = centroids
    
    for loop in range(self.max_iters): 
      clusters = defaultdict(list)
      sse = 0
      

      for point in data:
        temp_centroid = -1
        min_dist = 99999999
        for i, centroid in enumerate(self.init_centroids):
        
          d = self.d_func(point[:-1], centroid)
          
          if d < min_dist:
            temp_centroid = i
            min_dist = d
        
        clusters[temp_centroid].append(point)

      prev_centroids = self.init_centroids.copy()
      
      for key in clusters.keys():
        cluster = clusters[key]
        self.init_centroids[key] = calculate_centroid(cluster)

      if loop == 1 and self.show_first_centroid == True:
        print("Centroids after first iteration: ", self.init_centroids)

      if self.centroid_stop == True and self.init_centroids == prev_centroids:
        print("Centroids converged at iteration ",loop)
        break

      
      for key in clusters.keys():
        cluster = clusters[key]
        ce = self.init_centroids[key]

        for p in cluster:
          sse += euclidean_distance(ce, p)**2

      if self.show_sse == True and loop > 1 and self.sse_list[-1] <= sse:
        self.sse_list.pop()
        break

      self.sse_list.append(sse)
      print("Iteration number is ", loop," SSE: ",self.sse_list[-1])

    print("Time taken:", time.time() - start)
    print("Number of iterations:", loop)
    return [self.init_centroids, clusters]


In [ ]:
raw_data = loadCSV('/content/data.csv')


In [ ]:
raw_gth = loadCSV('/content/label.csv')

In [ ]:
type(raw_data[0])

In [ ]:
original_labels = dict(label_cluster(raw_data))

In [ ]:
original_labels[1]

In [ ]:
original_labels[1]=original_labels[1.0]

In [ ]:
kmeans = KMeans(max_iters=100)
[centroid_centers, clusters] = kmeans.fit(raw_data)

In [ ]:
clusters.keys()

In [ ]:
labels = {1: 0,2: 0,3: 0,4: 0,5: 0,6: 0,7: 0,8: 0,9:0}

for key in clusters:
  d = dict(label_cluster(clusters[key]))
  mx = 0
  s = 0
  label = ""
  for k in d:
    s += d[k]
    if d[k] > mx:
      mx = d[k]
      label = k
  labels[label] = mx



print("SSE =",kmeans.sse_list)
print("Original Labels: ", original_labels)
print("Predicted Labels: ", labels)

total = 0
mismatch = 0

for l in original_labels:
  total += original_labels[l]
  mismatch += abs(original_labels[l] - labels[l])

accuracy = (total - mismatch) / total

print("Accuracy =",accuracy)

In [ ]:
kmeans = KMeans(max_iters=100,d_func=cosine_similarity)
[centroid_centers, clusters] = kmeans.fit(raw_data)

labels = {1: 0,2: 0,3: 0,4: 0,5: 0,6: 0,7: 0,8: 0,9:0}

for key in clusters:
  d = dict(label_cluster(clusters[key]))
  mx = 0
  s = 0
  label = ""
  for k in d:
    s += d[k]
    if d[k] > mx:
      mx = d[k]
      label = k
  labels[label] = mx



print("SSE =",kmeans.sse_list)
print("Original Labels: ", original_labels)
print("Predicted Labels: ", labels)

total = 0
mismatch = 0

for l in original_labels:
  total += original_labels[l]
  mismatch += abs(original_labels[l] - labels[l])

accuracy = (total - mismatch) / total

print("Accuracy =",accuracy)

In [ ]:
kmeans = KMeans(max_iters=150,d_func=jaccard)
[centroid_centers, clusters] = kmeans.fit(raw_data)

labels = {1: 0,2: 0,3: 0,4: 0,5: 0,6: 0,7: 0,8: 0,9:0}

for key in clusters:
  d = dict(label_cluster(clusters[key]))
  mx = 0
  s = 0
  label = ""
  for k in d:
    s += d[k]
    if d[k] > mx:
      mx = d[k]
      label = k
  labels[label] = mx



print("SSE =",kmeans.sse_list)
print("Original Labels: ", original_labels)
print("Predicted Labels: ", labels)

total = 0
mismatch = 0

for l in original_labels:
  total += original_labels[l]
  mismatch += abs(original_labels[l] - labels[l])

accuracy = (total - mismatch) / total

print("Accuracy =",accuracy)

In [ ]:
kmeans = KMeans(max_iters=150,show_sse=True,centroid_stop=False)
[centroid_centers, clusters] = kmeans.fit(raw_data)
print(kmeans.sse_list)

In [ ]:
kmeans = KMeans(max_iters=100,d_func=cosine_similarity,show_sse=True,centroid_stop=False)
[centroid_centers, clusters] = kmeans.fit(raw_data)
print(kmeans.sse_list)

In [ ]:
kmeans = KMeans(max_iters=100,d_func=jaccard,show_sse=True,centroid_stop=False)
[centroid_centers, clusters] = kmeans.fit(raw_data)
print(kmeans.sse_list)

In [ ]:
kmeans = KMeans(max_iters=100,show_sse=False,centroid_stop=False)
[centroid_centers, clusters] = kmeans.fit(raw_data)
print(kmeans.sse_list)

In [ ]:
kmeans = KMeans(max_iters=100,d_func=cosine_similarity,show_sse=False,centroid_stop=False)
[centroid_centers, clusters] = kmeans.fit(raw_data)
print(kmeans.sse_list)

In [ ]:
kmeans = KMeans(max_iters=100,d_func=jaccard,show_sse=False,centroid_stop=False)
[centroid_centers, clusters] = kmeans.fit(raw_data)
print(kmeans.sse_list)